<a href="https://colab.research.google.com/github/doogzultz/time_series_project/blob/main/preprocessing_and_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from pandas.plotting import scatter_matrix
from tensorflow.keras.layers import Layer, Dense, Dropout, Input, \
Flatten, Concatenate, Conv1D, LSTM, Bidirectional, BatchNormalization, Activation, TimeDistributed, Lambda, MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.compose import ColumnTransformer
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

In [2]:
class g:
  window_size = 8
  batch_size = 16

In [3]:
url = "https://raw.githubusercontent.com/doogzultz/time_series_project/main/data.csv"
data = pd.read_csv(url)

In [4]:
data.columns

Index(['date', 'identifier', 'market_cap', 'sector', 'index_membership',
       'factor_1', 'factor_2', 'factor_3', 'factor_4', 'factor_5', 'factor_6',
       'factor_7', 'factor_8', 'factor_9', 'factor_10', 'target'],
      dtype='object')

In [5]:
data.drop(columns = 'index_membership', inplace = True)

In [6]:
data.sort_values(by = ['identifier', 'date'], inplace = True)
data

,date,identifier,market_cap,sector,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,factor_10,target
82,2010-01-06,AD41WBQFVG43,5.843968e+10,35,0.766639,-0.000032,0.832677,0.500608,0.193489,0.246089,0.753702,0.792736,-0.000005,0.109029,-0.000006
400,2010-01-13,AD41WBQFVG43,4.758865e+10,35,0.907349,-0.022933,0.346457,0.581694,0.648483,0.872304,0.723412,0.333230,-0.009504,0.121348,-0.008721
718,2010-01-20,AD41WBQFVG43,3.374757e+10,35,0.476465,-0.000114,0.592328,0.379747,0.810961,0.017737,0.619208,0.058918,-0.000099,0.890898,-0.000081
1036,2010-01-27,AD41WBQFVG43,4.908395e+10,35,0.265141,-0.000034,0.521405,0.682492,0.547349,0.155915,0.068966,0.699718,-0.000037,0.355059,-0.000018
1354,2010-02-03,AD41WBQFVG43,2.729189e+10,35,0.423987,0.000197,0.950500,0.628533,0.956287,0.426634,0.752799,0.961276,0.000163,0.196711,0.000318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12552,2010-10-06,ZMYXRS4KVOG2,6.504496e+08,25,0.069622,0.009936,0.588354,0.447469,0.353549,0.767779,0.790632,0.308049,0.012255,0.281193,0.005117
12862,2010-10-13,ZMYXRS4KVOG2,1.403710e+09,25,0.343444,0.002566,0.394802,0.143162,0.116762,0.744089,0.825323,0.559633,0.001058,0.755124,0.000446
13172,2010-10-20,ZMYXRS4KVOG2,7.284687e+08,25,0.857557,0.000146,0.892729,0.113346,0.574870,0.990100,0.319424,0.591153,0.000054,0.933679,0.000035
13483,2010-10-27,ZMYXRS4KVOG2,7.766592e+08,25,0.093894,-0.000885,0.311392,0.768537,0.589851,0.289180,0.431484,0.373209,-0.000411,0.225972,-0.000242


In [7]:
(data== 0).sum(axis=0)

date           0
identifier     0
market_cap     0
sector         0
factor_1       0
factor_2      68
factor_3       0
factor_4       0
factor_5       0
factor_6       0
factor_7       0
factor_8       0
factor_9      68
factor_10      0
target        68
dtype: int64

In [8]:
ident_counts= data.value_counts('identifier')
ident_counts

identifier
AD41WBQFVG43    104
UZ4DWDZ8ALZ4    104
KR47536Y10D4    104
KPJ8FTV9ESJ3    104
KMHQ727PU8E2    104
               ... 
PNUM57CYEB27      5
KHKNKSMXBWP3      5
LYCPQJ19AOY0      5
RENR64FT3I74      5
LALJLSN9UMP2      2
Length: 356, dtype: int64

In [9]:
data.value_counts('sector', normalize = True).sort_index()

sector
10    0.049221
15    0.093162
20    0.208527
25    0.132849
30    0.072759
35    0.049003
40    0.232812
45    0.051674
50    0.038352
55    0.071642
dtype: float64

In [10]:
data[data['identifier'].isin(list(ident_counts[ident_counts.values > 80].index))].value_counts('sector', normalize = True).sort_index()

sector
10    0.052423
15    0.093655
20    0.212044
25    0.126521
30    0.076618
35    0.045433
40    0.228443
45    0.052423
50    0.039048
55    0.073392
dtype: float64

In [11]:
data[data['identifier'].isin(list(ident_counts[ident_counts.values <= 80].index))].value_counts('sector', normalize = True).sort_index()

sector
10    0.010229
15    0.087152
20    0.165712
25    0.209902
30    0.025777
35    0.092471
40    0.286007
45    0.042553
50    0.029869
55    0.050327
dtype: float64

In [12]:
sector_onehot = pd.get_dummies(data['sector'], prefix='sector')
sector_onehot

,sector_10,sector_15,sector_20,sector_25,sector_30,sector_35,sector_40,sector_45,sector_50,sector_55
82,0,0,0,0,0,1,0,0,0,0
400,0,0,0,0,0,1,0,0,0,0
718,0,0,0,0,0,1,0,0,0,0
1036,0,0,0,0,0,1,0,0,0,0
1354,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
12552,0,0,0,1,0,0,0,0,0,0
12862,0,0,0,1,0,0,0,0,0,0
13172,0,0,0,1,0,0,0,0,0,0
13483,0,0,0,1,0,0,0,0,0,0


In [13]:
target = data['target']
data.drop(columns = ['sector', 'target'], inplace = True)
data = pd.concat([data, sector_onehot, target], axis = 1)
data

,date,identifier,market_cap,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,sector_15,sector_20,sector_25,sector_30,sector_35,sector_40,sector_45,sector_50,sector_55,target
82,2010-01-06,AD41WBQFVG43,5.843968e+10,0.766639,-0.000032,0.832677,0.500608,0.193489,0.246089,0.753702,...,0,0,0,0,1,0,0,0,0,-0.000006
400,2010-01-13,AD41WBQFVG43,4.758865e+10,0.907349,-0.022933,0.346457,0.581694,0.648483,0.872304,0.723412,...,0,0,0,0,1,0,0,0,0,-0.008721
718,2010-01-20,AD41WBQFVG43,3.374757e+10,0.476465,-0.000114,0.592328,0.379747,0.810961,0.017737,0.619208,...,0,0,0,0,1,0,0,0,0,-0.000081
1036,2010-01-27,AD41WBQFVG43,4.908395e+10,0.265141,-0.000034,0.521405,0.682492,0.547349,0.155915,0.068966,...,0,0,0,0,1,0,0,0,0,-0.000018
1354,2010-02-03,AD41WBQFVG43,2.729189e+10,0.423987,0.000197,0.950500,0.628533,0.956287,0.426634,0.752799,...,0,0,0,0,1,0,0,0,0,0.000318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12552,2010-10-06,ZMYXRS4KVOG2,6.504496e+08,0.069622,0.009936,0.588354,0.447469,0.353549,0.767779,0.790632,...,0,0,1,0,0,0,0,0,0,0.005117
12862,2010-10-13,ZMYXRS4KVOG2,1.403710e+09,0.343444,0.002566,0.394802,0.143162,0.116762,0.744089,0.825323,...,0,0,1,0,0,0,0,0,0,0.000446
13172,2010-10-20,ZMYXRS4KVOG2,7.284687e+08,0.857557,0.000146,0.892729,0.113346,0.574870,0.990100,0.319424,...,0,0,1,0,0,0,0,0,0,0.000035
13483,2010-10-27,ZMYXRS4KVOG2,7.766592e+08,0.093894,-0.000885,0.311392,0.768537,0.589851,0.289180,0.431484,...,0,0,1,0,0,0,0,0,0,-0.000242


In [14]:
import math
from math import floor as fl
ident_counts= data.value_counts('identifier')
def split_to_series(df, valid_size = .1, test_size = .05,length_cutoff = 3*g.window_size):
  ident_counts= df.value_counts('identifier')
  df_long = df[df['identifier'].isin(list(ident_counts[ident_counts.values > length_cutoff].index))]
  df_short = df[df['identifier'].isin(list(ident_counts[(ident_counts.values <= length_cutoff) & (ident_counts.values >= 2*g.window_size)].index))]
  series_dict = {}
  train_dict = {}
  valid_dict = {}
  test_dict = {}
  for i, ident in enumerate(df_long.value_counts('identifier').index):
    series_dict[i] = df_long[df_long['identifier'] == ident].sort_values('date').reset_index().drop(columns = 'index')
    train_dict[i] = series_dict[i].iloc[:fl((1-valid_size - test_size)*len(series_dict[i])) - 2*g.window_size]
    valid_dict[i] = series_dict[i].iloc[fl((1-valid_size - test_size)*len(series_dict[i]))- 2*g.window_size:fl((1- test_size)*len(series_dict[i]))- g.window_size]
    test_dict[i] = series_dict[i].iloc[fl((1- test_size)*len(series_dict[i]))- g.window_size:]
  for j, ident in enumerate(df_short.value_counts('identifier').index):
    train_dict[len(series_dict)+j] = df_short[df_short['identifier'] == ident].sort_values('date').reset_index().drop(columns = 'index')
  print(f"Number of time series to be trained and tested: {len([keys for keys in series_dict])}")
  print(f"Number of time series to be trained: {len([keys for keys in train_dict])}")
  return train_dict, valid_dict, test_dict

In [15]:
train_dict, valid_dict, test_dict = split_to_series(data)

Number of time series to be trained and tested: 331
Number of time series to be trained: 339


In [16]:
numerical_features = ['market_cap']+list(data.loc[:,'factor_1':'factor_10'].columns)+['target']
numerical_features

['market_cap',
 'factor_1',
 'factor_2',
 'factor_3',
 'factor_4',
 'factor_5',
 'factor_6',
 'factor_7',
 'factor_8',
 'factor_9',
 'factor_10',
 'target']

In [17]:
def preprocess(train_dict, valid_dict, test_dict, numerical_features):
  train_df = pd.concat(train_dict, ignore_index = True)
  t_mean = np.mean(train_df[numerical_features])
  t_std = np.std(train_df[numerical_features])
  for i in range(0,len(train_dict)):
    train_dict[i][numerical_features] = (train_dict[i][numerical_features]-t_mean)/t_std
  for i in range(0,len(valid_dict)):
    valid_dict[i][numerical_features] = (valid_dict[i][numerical_features]-t_mean)/t_std
    test_dict[i][numerical_features] = (test_dict[i][numerical_features]-t_mean)/t_std

  return train_dict, valid_dict, test_dict


In [18]:
train_scaled, valid_scaled, test_scaled = preprocess(train_dict, valid_dict, test_dict, numerical_features)

In [19]:
train_scaled[0]

,date,identifier,market_cap,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,sector_15,sector_20,sector_25,sector_30,sector_35,sector_40,sector_45,sector_50,sector_55,target
0,2010-01-06,AD41WBQFVG43,2.877332,0.921069,-0.043098,1.159976,0.003184,-1.061088,-0.886646,0.887815,...,0,0,0,0,1,0,0,0,0,-0.036569
1,2010-01-13,AD41WBQFVG43,2.212640,1.406296,-4.704516,-0.523708,0.284077,0.511038,1.294531,0.782325,...,0,0,0,0,1,0,0,0,0,-1.842457
2,2010-01-20,AD41WBQFVG43,1.364788,-0.079570,-0.059715,0.327696,-0.415492,1.072443,-1.682022,0.419427,...,0,0,0,0,1,0,0,0,0,-0.052204
3,2010-01-27,AD41WBQFVG43,2.304236,-0.808306,-0.043452,0.082105,0.633251,0.161593,-1.200734,-1.496849,...,0,0,0,0,1,0,0,0,0,-0.039049
4,2010-02-03,AD41WBQFVG43,0.969338,-0.260537,0.003507,1.567975,0.446332,1.574585,-0.257788,0.884670,...,0,0,0,0,1,0,0,0,0,0.030530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2011-04-20,AD41WBQFVG43,3.070243,-0.624454,-0.027959,-1.150029,-1.549021,1.629866,-0.291167,0.542785,...,0,0,0,0,1,0,0,0,0,-0.023964
68,2011-04-27,AD41WBQFVG43,1.902638,0.493737,-0.036856,1.645212,-0.746914,-1.393398,-0.710311,-0.683257,...,0,0,0,0,1,0,0,0,0,-0.037635
69,2011-05-04,AD41WBQFVG43,0.944957,-0.709339,-0.082636,-1.517489,-0.294331,-1.204468,-1.220560,1.595627,...,0,0,0,0,1,0,0,0,0,-0.052730
70,2011-05-11,AD41WBQFVG43,2.064631,-0.649902,-0.671639,-0.391335,-1.447296,-0.791861,1.339473,-0.244476,...,0,0,0,0,1,0,0,0,0,-2.092416


In [20]:
valid_scaled[0]

,date,identifier,market_cap,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,sector_15,sector_20,sector_25,sector_30,sector_35,sector_40,sector_45,sector_50,sector_55,target
72,2011-05-25,AD41WBQFVG43,1.802609,0.560933,0.780356,0.962631,0.762203,0.574268,0.461779,-1.365052,...,0,0,0,0,1,0,0,0,0,0.584909
73,2011-06-01,AD41WBQFVG43,1.643892,1.155420,-0.199619,-1.451153,0.063632,1.216239,-0.904808,0.217858,...,0,0,0,0,1,0,0,0,0,-0.484911
74,2011-06-08,AD41WBQFVG43,2.833501,-0.755184,-0.004197,0.228451,-0.359736,1.441092,1.278363,-0.187735,...,0,0,0,0,1,0,0,0,0,0.109609
75,2011-06-15,AD41WBQFVG43,3.174492,0.172864,-0.038757,0.371757,1.230699,-1.036347,-0.518142,0.689538,...,0,0,0,0,1,0,0,0,0,-0.036699
76,2011-06-22,AD41WBQFVG43,2.163958,1.580218,-0.158384,-1.395628,-0.589000,-1.156651,-0.769859,1.378711,...,0,0,0,0,1,0,0,0,0,-0.155451
77,2011-06-29,AD41WBQFVG43,2.965458,-0.186125,0.225054,-1.388665,0.114110,0.316904,1.414483,-1.435913,...,0,0,0,0,1,0,0,0,0,0.034606
78,2011-07-06,AD41WBQFVG43,3.429481,-0.993821,-0.054920,0.555808,0.340983,1.019601,1.336716,-0.328217,...,0,0,0,0,1,0,0,0,0,-0.743023
79,2011-07-13,AD41WBQFVG43,1.366409,1.684122,-0.049359,-1.718960,1.602827,0.697038,-1.257013,1.655672,...,0,0,0,0,1,0,0,0,0,-0.076231
80,2011-07-20,AD41WBQFVG43,2.105564,-0.970464,-1.092992,-0.036310,-0.182915,-0.710924,1.523046,0.049235,...,0,0,0,0,1,0,0,0,0,-1.623643
81,2011-07-27,AD41WBQFVG43,2.080937,0.987377,-0.572930,-1.559946,0.331276,1.086927,-0.229324,0.721965,...,0,0,0,0,1,0,0,0,0,-0.630055


In [21]:
test_scaled[0]

,date,identifier,market_cap,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,sector_15,sector_20,sector_25,sector_30,sector_35,sector_40,sector_45,sector_50,sector_55,target
90,2011-09-28,AD41WBQFVG43,1.590656,-0.477515,0.254152,-1.155218,1.209329,-1.318391,1.628659,-1.545261,...,0,0,0,0,1,0,0,0,0,1.536096
91,2011-10-05,AD41WBQFVG43,1.141237,1.622209,1.784764,-0.720812,0.992454,0.316789,-1.417923,-0.115184,...,0,0,0,0,1,0,0,0,0,0.761825
92,2011-10-12,AD41WBQFVG43,2.520157,0.526548,0.012644,0.824513,-0.987944,-0.098447,0.594118,-0.264213,...,0,0,0,0,1,0,0,0,0,0.014634
93,2011-10-19,AD41WBQFVG43,1.260228,0.707216,-0.013588,0.371809,-1.269664,0.630502,-0.811981,0.593789,...,0,0,0,0,1,0,0,0,0,0.023592
94,2011-10-26,AD41WBQFVG43,1.290237,-0.151598,-0.306641,-0.443944,0.311171,-0.792742,-1.400344,-1.555968,...,0,0,0,0,1,0,0,0,0,-0.356246
95,2011-11-02,AD41WBQFVG43,2.714305,0.179259,-0.024468,0.819282,1.298059,-1.504118,1.313307,0.857985,...,0,0,0,0,1,0,0,0,0,1.116552
96,2011-11-09,AD41WBQFVG43,2.231927,1.152144,-0.034073,-0.283000,0.691497,0.666868,-0.381691,1.541368,...,0,0,0,0,1,0,0,0,0,-0.009143
97,2011-11-16,AD41WBQFVG43,2.723592,1.617163,-0.075084,0.094701,0.988626,0.746851,0.863115,-0.822560,...,0,0,0,0,1,0,0,0,0,-0.220182
98,2011-11-23,AD41WBQFVG43,0.534226,1.020682,-0.021037,-0.012620,-1.592025,-0.092264,1.495839,0.719416,...,0,0,0,0,1,0,0,0,0,-0.013497
99,2011-11-30,AD41WBQFVG43,1.179641,1.004694,-0.037998,-0.873221,1.159124,0.033085,-0.561080,-0.523429,...,0,0,0,0,1,0,0,0,0,-0.036119


In [22]:
train_scaled[0].columns

Index(['date', 'identifier', 'market_cap', 'factor_1', 'factor_2', 'factor_3',
       'factor_4', 'factor_5', 'factor_6', 'factor_7', 'factor_8', 'factor_9',
       'factor_10', 'sector_10', 'sector_15', 'sector_20', 'sector_25',
       'sector_30', 'sector_35', 'sector_40', 'sector_45', 'sector_50',
       'sector_55', 'target'],
      dtype='object')

In [23]:
train_scaled[322].columns

Index(['date', 'identifier', 'market_cap', 'factor_1', 'factor_2', 'factor_3',
       'factor_4', 'factor_5', 'factor_6', 'factor_7', 'factor_8', 'factor_9',
       'factor_10', 'sector_10', 'sector_15', 'sector_20', 'sector_25',
       'sector_30', 'sector_35', 'sector_40', 'sector_45', 'sector_50',
       'sector_55', 'target'],
      dtype='object')

In [24]:
def drop_final_columns(data_dict):
  for i in range(0, len(data_dict)):
    data_dict[i].drop(columns = ['date', 'identifier'], inplace = True)
  return data_dict

In [25]:
train_scaled, valid_scaled, test_scaled = \
drop_final_columns(train_scaled), drop_final_columns(valid_scaled), drop_final_columns(test_scaled)

In [26]:
def plot_model_perf(history, metric = None):
  plt.figure(figsize=(12,8))
  epochs = range(len(history.history['loss']))
  plt.plot( epochs, history.history['loss'],color = 'blue', label = 'loss' )
  plt.plot(epochs, history.history['val_loss'], color = 'red', label = 'val_loss' )
  if metric:
    plt.plot( epochs, history.history[f'{metric}'],color = 'green', label = f'{metric}' )
    plt.plot(epochs, history.history[f'val_{metric}'], color = 'orange', label = f'val_{metric}' )
    plt.legend()
    plt.show()
  else:
    plt.legend()
    plt.show()

In [27]:
def convert_to_tensor(data_dict, window_size = g.window_size, batch_size = g.batch_size, shuffle_buffer = 1000):
    dataset = tf.data.Dataset.from_tensor_slices(np.asarray(data_dict[0], dtype= np.float32))
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.map(lambda window: (window[:-1], window[-1,-1]))
    for i in range(1, len(data_dict)):
      dataset_tmp = tf.data.Dataset.from_tensor_slices(np.asarray(data_dict[i], dtype= np.float32))
      dataset_tmp = dataset_tmp.window(window_size + 1, shift=1, drop_remainder=True)
      dataset_tmp = dataset_tmp.flat_map(lambda window: window.batch(window_size + 1))
      dataset_tmp = dataset_tmp.map(lambda window: (window[:-1], window[-1,:-1], window[-1,-1]))
      dataset = dataset.concatenate(dataset_tmp)
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [28]:
train = convert_to_tensor(train_scaled)
valid = convert_to_tensor(valid_scaled)
test = convert_to_tensor(test_scaled)

In [29]:
import tensorflow.keras.backend as K

def smape_loss(y_true, y_pred):
    epsilon = 1e-4
    #summ = K.maximum(K.abs(y_true) + K.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = K.abs(y_pred - y_true) / (K.abs(y_true) + K.abs(y_pred) + epsilon) * 2.0
    return smape

In [30]:
def conv_block(input_,filter_units,kernel_size,dr):
    x = Conv1D(filters = filter_units/2, kernel_size = kernel_size, strides = 1,
                                                              kernel_initializer='he_normal', padding = 'causal',activation = 'relu')(input_)
    x = Dropout(dr)(x)
    x = MaxPooling1D()(x)
    x = Conv1D(filters = filter_units, kernel_size = kernel_size, strides = 1,
                                                              kernel_initializer='he_normal', padding = 'causal',activation = 'relu')(x)
    x = Dropout(dr)(x)
    out = MaxPooling1D()(x)
    return out

In [31]:
def lstm_block(input_,lstm_units, dr):
    x = LSTM(lstm_units*2, return_sequences = True, dropout = dr)(input_)
    out = LSTM(lstm_units, return_sequences = False, dropout = dr)(x)
    return(out)

In [32]:
def dense_block(input_, dense_units, dr):
    x = Dense(dense_units, kernel_initializer="he_normal", activation = 'relu')(input_)
    x = Dropout(dr)(x)
    x = Dense(dense_units, kernel_initializer="he_normal", activation = 'relu')(x)
    x = Dropout(dr)(x)
    out = Dense(1)(x)
    return out

In [40]:
tf.keras.backend.clear_session()

def create_time_model(filter_units = 64, lstm_units = 32, kernel_size = 3, dense_units = 128, dr = .2 ):

    input_ = Input(shape=(None,22))
    input_2 = Input(shape = (None, None, 21))
    x = conv_block(input_,filter_units, kernel_size, dr)                                                            
    x = lstm_block(x, lstm_units, dr)
    x = Concatenate([x, input_2])
    out = dense_block(x, dense_units, dr)
    model = Model(inputs = input_, outputs = out)

    return model

model = create_time_model()
model.compile(loss = 'mape', optimizer = 'adam', metrics = ['mae', 'mape'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 22)]        0         
                                                                 
 conv1d (Conv1D)             (None, None, 32)          2144      
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 32)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 64)          6208      
                                                                 
 dropout_1 (Dropout)         (None, None, 64)          0         
                                                             

In [ ]:
early_stopping = EarlyStopping(patience = 7, restore_best_weights= True)
history = model.fit(train, epochs = 30, validation_data = valid, callbacks = early_stopping)
plot_model_perf(history, ['mape', 'mae'])
model.evaluate(valid)

Epoch 1/30
1197/1197 [==============================] - 33s 25ms/step - loss: 162.5069 - mae: 0.3752 - mape: 162.5069 - val_loss: 110.9699 - val_mae: 0.6270 - val_mape: 110.9699
Epoch 2/30
1197/1197 [==============================] - 34s 28ms/step - loss: 127.9256 - mae: 0.3727 - mape: 127.9256 - val_loss: 106.1352 - val_mae: 0.6291 - val_mape: 106.1352
Epoch 3/30
1197/1197 [==============================] - 39s 32ms/step - loss: 114.0630 - mae: 0.3726 - mape: 114.0630 - val_loss: 104.8473 - val_mae: 0.6274 - val_mape: 104.8473
Epoch 4/30
1197/1197 [==============================] - 33s 28ms/step - loss: 107.1905 - mae: 0.3732 - mape: 107.1905 - val_loss: 102.4575 - val_mae: 0.6277 - val_mape: 102.4575
Epoch 5/30
1197/1197 [==============================] - 40s 33ms/step - loss: 106.9029 - mae: 0.3725 - mape: 106.9029 - val_loss: 102.4929 - val_mae: 0.6277 - val_mape: 102.4929
Epoch 6/30
1197/1197 [==============================] - 29s 24ms/step - loss: 104.6450 - mae: 0.3727 - mape: 1

In [ ]:
model.evaluate(test)

In [ ]:
xbatch, ybatch = iter(test).get_next()

In [ ]:
xbatch

In [ ]:
model.predict(xbatch)

In [ ]:
ybatch